In [1]:
import numpy as np
from dtw import dtw

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
def normalized_dtw_distance(series1: np.ndarray, series2: np.ndarray) -> float:
    """
    Computes the normalized DTW distance between two time series.

    This function calculates the DTW distance between two input time series
    and normalizes it by the length of the longer series to provide a
    length-invariant measure of similarity.

    Parameters:
    series1 (np.ndarray): The first time series.
    series2 (np.ndarray): The second time series.

    Returns:
    float: The normalized DTW distance.
    """
    # Compute DTW distance using the dtw package
    alignment = dtw(series1, series2, keep_internals=True)
    dtw_distance = alignment.distance

    # Determine the length of the longer time series
    max_length = max(len(series1), len(series2))

    # Normalize the DTW distance by the length of the longer series
    normalized_distance = dtw_distance / max_length

    return normalized_distance


In [3]:
# Example time series data
ts1 = np.array([1, 2, 3, 4, 5])
ts2 = np.array([2, 3, 4])

# Compute normalized DTW distance
distance = normalized_dtw_distance(ts1, ts2)
print(f"Normalized DTW Distance: {distance}")


Normalized DTW Distance: 0.4


In [4]:
# Example time series data
ts1 = np.array([1, 2, 3, 4, 5])
ts2 = np.array([2, 10, 4])

# Compute normalized DTW distance
distance = normalized_dtw_distance(ts1, ts2)
print(f"Normalized DTW Distance: {distance}")

Normalized DTW Distance: 2.0
